In [ ]:
import json
import numpy as np
from PIL import Image
import viser

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] WebRTC GUI backend enabled.


RuntimeError: [1;31m[Open3D Error] (void open3d::visualization::gui::Application::SetWindowSystem(std::shared_ptr<open3d::visualization::gui::WindowSystem>)) /root/Open3D/cpp/open3d/visualization/gui/Application.cpp:348: Cannot set WindowSystem. It is already set.
[0;m

In [2]:
IMAGE_PATH = "/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/camera_0/000000.jpg"
DEPTH_PATH = "/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/depth/000000.png"

In [3]:
camera_info = json.loads(open("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston/cameras/105972988238547_intrinsics.json").read())

In [ ]:
# Load RGB and depth images
color = np.array(Image.open(IMAGE_PATH))
depth = np.array(Image.open(DEPTH_PATH))

# Get camera intrinsics
focal_length = camera_info["focal_length_pixels"]
width = camera_info["width"]
height = camera_info["height"]
cx = width / 2
cy = height / 2

# Create pixel grid
h, w = depth.shape[:2]
i, j = np.meshgrid(np.arange(w), np.arange(h), indexing='xy')

# Unproject to 3D
z = depth.astype(np.float32) / 1000.0  # Convert to meters (adjust scale as needed)
x = (i - cx) * z / focal_length
y = (j - cy) * z / focal_length

# Stack to get 3D points
points = np.stack([x, y, z], axis=-1).reshape(-1, 3)

# Get RGB colors (normalized to 0-1)
colors = color.reshape(-1, 3) / 255.0

# Filter out invalid points (where depth is 0)
valid_mask = (z.reshape(-1) > 0)
points = points[valid_mask]
colors = colors[valid_mask]

# Apply coordinate transform (same as open3d)
# [1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]
points[:, 1] = -points[:, 1]
points[:, 2] = -points[:, 2]

# Visualize with viser
server = viser.ViserServer()
server.scene.add_point_cloud(
    name="rgbd_pointcloud",
    points=points,
    colors=colors,
    point_size=0.01
)

print(f"Point cloud created with {len(points)} points")
print(f"Server running at http://localhost:8080")

[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [5]:
color.get_max_bound(), depth.get_max_bound()

(array([3840., 2160.]), array([3840., 2160.]))

In [1]:
from pathlib import Path
import os

In [1]:
#!/usr/bin/env python3
"""
Combine all masks from filtered_masks subfolders into single masks.

For each sequence in boston_cusfm_sequence:
- Read all masks from filtered_masks/000000/, 000001/, etc.
- For each frame (000000, 000001, etc.), combine all individual masks (car_0.png, car_1.png, etc.)
- Save the combined mask as masks/000000.png, masks/000001.png, etc.
"""

import os
from pathlib import Path
import numpy as np
from PIL import Image
from tqdm import tqdm
from tqdm.auto import tqdm


def combine_masks_for_frame(frame_folder):
    """
    Combine all mask images in a frame folder into a single mask.
    
    Args:
        frame_folder: Path to folder containing individual masks (e.g., car_0.png, car_1.png)
    
    Returns:
        Combined mask as numpy array (binary: 0 or 255)
    """
    mask_files = sorted(list(Path(frame_folder).glob("*.png")))
    
    if not mask_files:
        return None
    
    # Read first mask to get dimensions
    first_mask = np.array(Image.open(mask_files[0]))
    combined_mask = np.zeros_like(first_mask, dtype=np.uint8)
    
    # Combine all masks using logical OR
    for mask_file in mask_files:
        mask = np.array(Image.open(mask_file))
        # Convert to binary if needed
        if mask.ndim == 3:
            mask = mask[:, :, 0]  # Take first channel if RGB
        combined_mask = np.maximum(combined_mask, mask)
    
    return combined_mask


def process_sequence(sequence_path):
    """
    Process a single sequence: combine all filtered masks and save to masks folder.
    
    Args:
        sequence_path: Path to sequence folder
    """
    filtered_masks_path = Path(sequence_path) / "filtered_masks"
    masks_output_path = Path(sequence_path) / "masks"
    
    # Check if filtered_masks exists
    if not filtered_masks_path.exists():
        print(f"  ⚠️  No filtered_masks folder found in {sequence_path.name}")
        return
    
    # Get all frame folders (000000, 000001, etc.)
    frame_folders = sorted([f for f in filtered_masks_path.iterdir() if f.is_dir()])
    
    if not frame_folders:
        print(f"  ⚠️  No frame folders found in filtered_masks for {sequence_path.name}")
        return
    
    # Create masks output folder
    masks_output_path.mkdir(exist_ok=True)
    
    print(f"  Processing {len(frame_folders)} frames...")
    
    # Process each frame
    for frame_folder in tqdm(frame_folders, desc=f"  {sequence_path.name}"):
        frame_name = frame_folder.name
        
        # Combine all masks in this frame folder
        combined_mask = combine_masks_for_frame(frame_folder)
        
        if combined_mask is not None:
            # Save combined mask
            output_path = masks_output_path / f"{frame_name}.jpg.png"
            Image.fromarray(combined_mask).save(output_path)
        else:
            print(f"    ⚠️  No masks found for frame {frame_name}")
    
    print(f"  ✓ Saved {len(frame_folders)} combined masks to {masks_output_path}")


def main():
    # Path to all sequences
    base_path = Path("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_cusfm_sequence")
    
    # Get all sequence folders (folders with pattern *_*)
    sequence_folders = sorted([f for f in base_path.iterdir() 
                              if f.is_dir() and '_' in f.name and f.name[0].isdigit()])
    
    if not sequence_folders:
        print("No sequence folders found!")
        return
    
    print(f"Found {len(sequence_folders)} sequences to process\n")
    
    # Process each sequence
    for sequence_folder in sequence_folders:
        print(f"Processing: {sequence_folder.name}")
        process_sequence(sequence_folder)
        print()
    
    print("✓ All sequences processed!")

In [1]:
3840/2, 2160/2

(1920.0, 1080.0)

In [ ]:
# if __name__ == "__main__":
main()

In [ ]:
# Data Preperation commands
export BASE_PATH=data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/camera_0

python filter_sequence_by_motion.py ${BASE_PATH} --mask-dir ${BASE_PATH}_masks --motion-threshold 10 --auxiliary-dir ${BASE_PATH}_masks

In [ ]:
colmap feature_extractor \
    --database_path $DB_PATH \
    --image_path $IMAGE_PATH \
    --ImageReader.single_camera 1 \
    --ImageReader.camera_model SIMPLE_PINHOLE \
    --ImageReader.camera_params "1800.161857181376,1920.0,1080.0" \
    --SiftExtraction.use_gpu 1

colmap feature_extractor --database_path "${PROJECT_PATH}/database.db" --ImageReader.camera_model SIMPLE_PINHOLE --image_path "${PROJECT_PATH}/images" --ImageReader.mask_path "${PROJECT_PATH}/masks_filtered" --ImageReader.single_camera 1 --SiftExtraction.use_gpu 1


colmap mapper \
    --database_path $DB_PATH \
    --image_path $IMAGE_PATH \
    --output_path "${PROJECT_PATH}/sparse"

colmap mapper \
    --database_path $DB_PATH \
    --image_path $IMAGE_PATH \
    --output_path "${PROJECT_PATH}/sparse" \
    --Mapper.ba_refine_principal_point 1 \
    --Mapper.ba_refine_extra_params 1

colmap bundle_adjuster \
    --input_path "${PROJECT_PATH}/sparse/0" \
    --output_path "${PROJECT_PATH}/sparse/0" \
    --BundleAdjustment.refine_focal_length 0 \
    --BundleAdjustment.refine_principal_point 0 \
    --BundleAdjustment.refine_extra_params 0

ns-process-data images --data $IMAGE_PATH \
    --output-dir $NERFSTUDIO_DATA \
    --skip-colmap \
    --colmap-model-path "${PROJECT_PATH}/sparse/0"

In [ ]:
colmap feature_extractor --database_path "data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/database.db" --ImageReader.camera_model SIMPLE_PINHOLE --image_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/camera_0_filtered --ImageReader.mask_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/masks_filtered --ImageReader.single_camera 1 --SiftExtraction.use_gpu 1

colmap exhaustive_matcher --database_path "data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/database.db" --SiftMatching.use_gpu 1

colmap sequential_matcher --database_path "data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/database.db" --SiftMatching.use_gpu 1 --SequentialMatching.overlap 15 --SequentialMatching.loop_detection 0

colmap mapper --database_path "data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/database.db" --image_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/camera_0_filtered --output_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/sparse

colmap model_converter --input_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/sparse/0 --output_path data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/sparse/0 --output_type TXT

In [ ]:
colmap feature_extractor --database_path "data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/database.db" --ImageReader.camera_model SIMPLE_PINHOLE --image_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/camera_0_filtered --ImageReader.mask_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/masks_filtered --ImageReader.single_camera 1 --SiftExtraction.use_gpu 1

colmap exhaustive_matcher --database_path "data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/database.db" --SiftMatching.use_gpu 1

colmap sequential_matcher --database_path "data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/database.db" --SiftMatching.use_gpu 1 --SequentialMatching.overlap 15 --SequentialMatching.loop_detection 0

colmap mapper --database_path "data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/database.db" --image_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/camera_0_filtered --output_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/sparse

colmap model_converter --input_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/sparse/0 --output_path data/boston_cusfm_sequence/0030_g1uEFitJ8hRXF0USBhF--Q/sparse/0 --output_type TXT

In [ ]:
colmap feature_extractor --database_path "data/boston/database.db" --ImageReader.camera_model SIMPLE_PINHOLE --image_path data/boston/images --ImageReader.single_camera 0 --SiftExtraction.use_gpu 1

colmap vocab_tree_builder --database_path "data/boston/database.db" --vocab_tree_path "data/boston/vocab_tree.bin"

colmap vocab_tree_matcher \
    --database_path "data/boston/database.db" \
    --SiftMatching.use_gpu 1 \
    --VocabTreeMatching.vocab_tree_path "data/vocab_tree_flickr100K_words256K.bin"

colmap exhaustive_matcher --database_path "data/boston/database.db" --SiftMatching.use_gpu 1

colmap sequential_matcher --database_path "data/boston/database.db" --SiftMatching.use_gpu 1 --SequentialMatching.overlap 15 --SequentialMatching.loop_detection 0

colmap mapper --database_path "data/boston/database.db" --image_path data/boston/images --output_path data/boston/sparse

colmap model_converter --input_path data/boston/sparse/0 --output_path data/boston/sparse/0 --output_type TXT

In [ ]:
export IMAGES_DIR="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/images"
export OUTPUTS_DIR="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/hloc_outputs"
export MASKS_DIR="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/masks_filtered_colmap"
export SFM_DIR="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/sparse"

export IMAGES_DIR="data/boston_custom_data/images"
export OUTPUTS_DIR="data/boston_custom_data/hloc_outputs"
export MASKS_DIR="data/boston_custom_data/masks_filtered_colmap"
export SFM_DIR="data/boston_custom_data/sparse"

python -m hloc.extract_features \
    --conf netvlad \
    --image_dir $IMAGES_DIR \
    --export_dir $OUTPUTS_DIR

python -m hloc.extract_features \
    --conf superpoint_aachen \
    --image_dir $IMAGES_DIR \
    --export_dir $OUTPUTS_DIR

python -m hloc.pairs_from_retrieval \
    --descriptors $OUTPUTS_DIR/global-feats-netvlad.h5 \
    --output $OUTPUTS_DIR/pairs-retrieval.txt \
    --num_matched 20

python -m hloc.match_dense \
    --conf "loftr" \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --matches $OUTPUTS_DIR/matches.h5 --export_dir $OUTPUTS_DIR \
    --image_dir $IMAGES_DIR

python -m hloc.reconstruction \
    --sfm_dir $SFM_DIR \
    --image_dir $IMAGES_DIR \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --camera_mode "PER_IMAGE" \
    --features $OUTPUTS_DIR/feats_matches-loftrmatches-loftr.h5 \
    --matches $OUTPUTS_DIR/matches.h5 \
    --image_options "camera_model=PINHOLE" \
    --image_options "mask_path=$MASKS_DIR"

python -m hloc.reconstruction \
    --sfm_dir $SFM_DIR \
    --image_dir $IMAGES_DIR \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --camera_mode "SINGLE" \
    --features $OUTPUTS_DIR/feats_matches-loftrmatches-loftr.h5 \
    --matches $OUTPUTS_DIR/matches.h5 \
    --image_options camera_model='"PINHOLE"' mask_path='"$MASKS_DIR"' camera_params='"1800.161857181376,1800.161857181376,1920.0,1080.0"'

In [ ]:
# NerfStudio Processing & Training

export NERFSTUDIO_DATA="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/nerfstudio_data"
export MASKS_DIR="data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/masks_filtered"

ns-process-data images \
    --data $IMAGES_DIR \
    --output-dir $NERFSTUDIO_DATA \
    --matching-method sequential --camera-type perspective --sfm-tool hloc --feature-type superpoint --matcher-type superglue --num-downscales 0 --use-sfm-depth --use-single-camera-mode


ns-train splatfacto --data $NERFSTUDIO_DATA \
    --output-dir data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/nerfstudio_output \
    --mixed-precision True \
    --max-num-iterations 50000 \
    
ns-train splatfacto colmap --colmap-path "${SFM_DIR}" \
    --images-path "${IMAGES_DIR}" \
    --masks-path "${MASKS_DIR}" \
    --max-2D-matches-per-3D-point -1
    --downscale-factor 1

In [5]:
# Understaing Mast3r & Depth Anything depths

import numpy as np

da2_depth = np.load("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/depths_npy/000001.npy")
mast3r_depth = np.load("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_cusfm_sequence/0000_zUKZjC0qOqVkYWxD-1n6EA/depths_mast3r//000001.npy")

In [6]:

da2_depth.max(), da2_depth.min(), mast3r_depth.max(), mast3r_depth.min()

(np.float32(395.94016),
 np.float32(-95.247734),
 np.float32(259.66083),
 np.float32(0.83021694))

In [7]:
(da2_depth[da2_depth<0]).shape

(35132,)

In [9]:
2160 * 3840

8294400

In [25]:
disp_min = 1/0.1
disp_max = 1/200

output_norm = (da2_depth - da2_depth.min()) / (da2_depth.max() - da2_depth.min())

inverse_depth_map = output_norm * (disp_min - disp_max) + disp_max

disp_metric = disp_min + da2_depth * (disp_max - disp_min)
depth_metric = 1 / disp_metric
depth_metric.max(), depth_metric.min()

(np.float32(127.22349), np.float32(-75.29628))

In [2]:
# Combine multiple folders with same image names but different images into a single folder.
from pathlib import Path
import shutil
import os

BASE_PATH = Path("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_custom_data/gathered_data")
FINAL_PATH = Path("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/boston_custom_data/masks")

idx = 0
for img_path in BASE_PATH.glob("camera_*_filtered/*.png"):
    dest_path = FINAL_PATH / f"{idx:06d}.png"
    shutil.copy(img_path, dest_path)
    idx += 1

In [ ]:
export IMAGES_DIR="data/boston_custom_data/images_filtered"
export OUTPUTS_DIR="data/boston_custom_data/hloc_outputs"
export MASKS_DIR="data/boston_custom_data/masks_filtered"
export SFM_DIR="data/boston_custom_data/sparse"

# export IMAGES_DIR="data/boston_custom_data/images"
# export OUTPUTS_DIR="data/boston_custom_data/hloc_outputs"
# export MASKS_DIR="data/boston_custom_data/masks_filtered_colmap"
# export SFM_DIR="data/boston_custom_data/sparse"

python -m hloc.extract_features \
    --conf netvlad \
    --image_dir $IMAGES_DIR \
    --export_dir $OUTPUTS_DIR

python -m hloc.extract_features \
    --conf superpoint_aachen \
    --image_dir $IMAGES_DIR \
    --export_dir $OUTPUTS_DIR

python -m hloc.pairs_from_retrieval \
    --descriptors $OUTPUTS_DIR/global-feats-netvlad.h5 \
    --output $OUTPUTS_DIR/pairs-retrieval.txt \
    --num_matched 20

python -m hloc.pairs_from_exhaustive \
    --output $OUTPUTS_DIR/pairs-exhaustive.txt \
    --features $OUTPUTS_DIR/global-feats-netvlad.h5 \
    --image_list $IMAGES_DIR


python -m hloc.match_dense \
    --conf "loftr" \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --matches $OUTPUTS_DIR/matches.h5 --export_dir $OUTPUTS_DIR \
    --image_dir $IMAGES_DIR

python -m hloc.reconstruction \
    --sfm_dir $SFM_DIR \
    --image_dir $IMAGES_DIR \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --camera_mode "PER_IMAGE" \
    --features $OUTPUTS_DIR/feats_matches-loftrmatches-loftr.h5 \
    --matches $OUTPUTS_DIR/matches.h5 \
    --image_options "camera_model=PINHOLE" \
    --image_options "mask_path=$MASKS_DIR"

python -m hloc.reconstruction \
    --sfm_dir $SFM_DIR \
    --image_dir $IMAGES_DIR \
    --pairs $OUTPUTS_DIR/pairs-retrieval.txt \
    --camera_mode "PER_IMAGE" \
    --features $OUTPUTS_DIR/feats_matches-loftrmatches-loftr.h5 \
    --matches $OUTPUTS_DIR/matches.h5 \
    --image_options camera_model='"PINHOLE"' mask_path='"$MASKS_DIR"' camera_params='"1800.161857181376,1800.161857181376,1920.0,1080.0"'

In [ ]:
colmap feature_extractor --database_path "data/boston_custom_data/database.db" --ImageReader.camera_model SIMPLE_PINHOLE --image_path data/boston_custom_data/images_filtered --ImageReader.mask_path data/boston_custom_data/masks_filtered --ImageReader.single_camera 1 --SiftExtraction.use_gpu 1

colmap exhaustive_matcher --database_path "data/boston_custom_data/database.db" --SiftMatching.use_gpu 1

colmap sequential_matcher --database_path "data/boston_custom_data/database.db" --SiftMatching.use_gpu 1 --SequentialMatching.overlap 15 --SequentialMatching.loop_detection 0

colmap mapper --database_path "data/boston_custom_data/database.db" --image_path data/boston_custom_data/camera_0_filtered --output_path data/boston_custom_data/sparse

colmap model_converter --input_path data/boston_custom_data/sparse/0 --output_path data/boston_custom_data/sparse/0 --output_type TXT

In [2]:
import pycolmap
import rerun as rr  # NOTE: `rerun`, not `rerun-sdk`!
import numpy as np
from pathlib import Path
from PIL import Image

In [3]:
# rr.spawn(server_memory_limit="16GB")
rr.init("Hloc Reconstruction Explorer")
rr.connect_grpc()

In [4]:
BASE_PATH = Path("/home/vamsik1211/Data/Projects/3D-Reconstructions/CityScapeGS/data/Urban3D-Dataset/ArtSci/Smith et al/ArtSci_fine")
RECONSTRUCTION_PATH = BASE_PATH / "hloc_outputs_single/sfm"
DB_PATH = BASE_PATH / "hloc_outputs_single/sfm/database.db"
IMAGES_PATH = BASE_PATH / "images_combined"

model = pycolmap.Reconstruction(str(RECONSTRUCTION_PATH))
database = pycolmap.Database()

In [5]:
cameras = model.cameras
frames = model.frames
images = model.images

In [6]:
points = model.points3D

In [ ]:
points3D, colors, radii = [], [], []

for key, point in points.items():
    point3D = point.xyz
    color = point.color / 255.0  # Normalize color to [0, 1]
    points3D.append(point3D)
    colors.append(color)
    radii.append(0.0005)

    # rr.log_point(
    #     f"point_cloud/point_{key}",
    #     point3D,
    #     color=color,
    #     radius=0.05
    # )

points3D = np.array(points3D)
colors = np.array(colors)

# points_rr = rr.Points3D(
#     positions=points3D,
#     colors=colors
# )
# rr.log(f"point_cloud/points", points_rr)

In [9]:
points3D.max(), points3D.min()

(np.float64(102.59324865114051), np.float64(-193.82924940047863))

In [5]:
for idx, (key, image) in enumerate(images.items()):
    homo_mat = image.cam_from_world().matrix()
    image_name = image.name
    image_path = IMAGES_PATH / image_name
    image_pil = Image.open(image_path).convert("RGB")
    
    if image_path.exists():
        rr.log(f"images/{image_name}", rr.Image(np.asarray(image_pil), color_model=rr.ColorModel.RGB))
    else:
        print(f"Image not found: {image_path}")
        rr.log(f"Image not found: {image_path}")

    if idx > 20:
        break
    

In [50]:
image.name

'cam0_DJI_0749.JPG'

In [45]:
transforms.matrix()

array([[ 0.81634227, -0.2739448 ,  0.50846785,  4.98760584],
       [ 0.37298625,  0.92221666, -0.10196904,  3.58788728],
       [-0.44098364,  0.27289315,  0.85502208, -2.06016541]])

In [8]:
db = database.open(str(DB_PATH))

In [22]:
cameras = db.read_all_cameras()
frames = db.read_all_frames()
images = db.read_all_images()

In [15]:
frames

[Frame(frame_id=1, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 1)]),
 Frame(frame_id=2, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 2)]),
 Frame(frame_id=3, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 3)]),
 Frame(frame_id=4, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 4)]),
 Frame(frame_id=5, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 5)]),
 Frame(frame_id=6, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 6)]),
 Frame(frame_id=7, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 7)]),
 Frame(frame_id=8, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 8)]),
 Frame(frame_id=9, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 9)]),
 Frame(frame_id=10, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 10)]),
 Frame(frame_id=11, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 11)]),
 Frame(frame_id=12, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 12)]),
 Frame(frame_id=13, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 13)]),
 Frame(frame_id=14, rig_id=1, has_pose=0, data_ids=[(CAMERA, 1, 14)]),
 Frame(frame_id=15, rig_